In [ ]:
import tensorflow as tf
import subprocess
from datetime import datetime

In [ ]:
class GPUMemoryManager:
    def __init__(self, memory_fraction=0.5, allow_growth=True):
        self.memory_fraction = memory_fraction
        self.allow_growth = allow_growth
        self.gpus = tf.config.experimental.list_physical_devices('GPU')
        
    def get_total_gpu_memory(self):
        try:
            output = subprocess.check_output(
                ['nvidia-smi', '--query-gpu=memory.total', '--format=csv,nounits,noheader'],
                encoding='utf-8'
            )
            return int(output.strip())
        except:
            return None

    def configure(self):
        if not self.gpus:
            print("GPU не знайдено")
            return False

        try:
            total_memory = self.get_total_gpu_memory()
            if total_memory is None:
                print("Не вдалося отримати інформацію про пам'ять GPU")
                return False

            memory_limit = int(total_memory * self.memory_fraction)

            for gpu in self.gpus:
                tf.config.experimental.set_memory_growth(gpu, self.allow_growth)
                
                tf.config.experimental.set_virtual_device_configuration(
                    gpu,
                    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]
                )
            
            print(f"GPU налаштовано: ліміт {memory_limit}MB ({self.memory_fraction*100}%)")
            return True

        except RuntimeError as e:
            print(f"Помилка конфігурації GPU: {e}")
            return False

In [ ]:
gpu_manager = GPUMemoryManager(memory_fraction=0.5, allow_growth=True)
if not gpu_manager.configure():
    print("Помилка налаштування GPU. Вихід")

GPU налаштовано: ліміт 4096MB (50.0%)


In [85]:
class CustomTrainingMonitor:
    def __init__(self, log_frequency=10):
        self.log_frequency = log_frequency
        self.peak_memory = 0
        self.start_time = None
        self.current_epoch = 0
        self.current_batch = 0

    def get_detailed_memory_info(self):
        memory = tf.config.experimental.get_memory_info('GPU:0')
        return {
            'tf_current': memory['current'] / (1024 * 1024),
            'tf_peak': memory['peak'] / (1024 * 1024)
        }

    def print_memory_stats(self):
        memory = self.get_detailed_memory_info()
        if memory:
            print(f"TensorFlow reported memory: {memory['tf_current']:.0f}MB (Peak: {memory['tf_peak']:.0f}MB)")
            self.peak_memory = max(self.peak_memory, memory['tf_current'])
            
    def on_training_start(self):
        self.start_time = datetime.now()
        print("Running training...")
        
    def on_epoch_start(self):
        self.current_epoch += 1
        self.current_batch = 0
        print(f"\nEpoch {self.current_epoch}")
        
        print("Epoch start: ", end="")
        self.print_memory_stats()
        
    def on_batch_end(self, loss, accuracy):
        self.current_batch += 1
        if self.current_batch % self.log_frequency == 0:
            print(f"Batch {self.current_batch} - Loss: {loss:.4f} - Accuracy: {accuracy:.4f}")
            self.print_memory_stats()

    def on_epoch_end(self, train_loss, train_accuracy, valid_loss, valid_accuracy):
        print(f"\nTrain Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f}")
        print(f"Valid Loss: {valid_loss:.4f} - Valid Accuracy: {valid_accuracy:.4f}")
        print("Epoch end: ", end="")
        self.print_memory_stats()

    def on_training_end(self):
        duration = datetime.now() - self.start_time
        print("\nEnd of training...")
        print(f"Total duration: {duration}")

In [ ]:
from datasets import load_dataset

In [6]:
dataset = load_dataset("bentrevett/multi30k")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [ ]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

In [ ]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [ ]:
import spacy

In [11]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [12]:
string = "What a lovely day it is today!, submissively"
print([token.text for token in en_nlp.tokenizer(string)])
print([token.is_stop for token in en_nlp.tokenizer(string)])

string = 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'
print([token.text for token in de_nlp.tokenizer(string)])
print([token.is_stop for token in de_nlp.tokenizer(string)])

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!', ',', 'submissively']
[True, True, False, False, True, True, False, False, False, False]
['Zwei', 'junge', 'weiße', 'Männer', 'sind', 'im', 'Freien', 'in', 'der', 'Nähe', 'vieler', 'Büsche', '.']
[True, False, False, False, True, True, False, True, True, False, False, False, False]


In [13]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]

    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]

    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]

    return {"en_tokens": en_tokens, "de_tokens": de_tokens} 

In [14]:
max_length = 1000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token
}

In [15]:
train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

In [16]:
{key: train_data[0][key] for key in ["en_tokens", "de_tokens"]}

{'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [17]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
special_tokens = [unk_token, pad_token, sos_token, eos_token]

In [18]:
from tokenizers import Tokenizer

In [ ]:
from collections import Counter

In [ ]:
from tokenizers.models import WordLevel

In [ ]:
def build_vocab(tokenized_sentences, min_freq, special_tokens):
    vocab = {token: idx for idx, token in enumerate(special_tokens)}

    filtered_sentences = [
        [word for word in sentence if word not in ["<sos>", "<eos>"]]
        for sentence in tokenized_sentences
    ]

    word_counts = Counter(word for sentence in filtered_sentences for word in sentence)

    for word, count in word_counts.items():
        if count >= min_freq:
            vocab[word] = len(vocab)
    
    return vocab

In [ ]:
en_vocab = build_vocab(train_data["en_tokens"], min_freq, special_tokens)
de_vocab = build_vocab(train_data["de_tokens"], min_freq, special_tokens)

In [44]:
len(en_vocab), len(de_vocab)

(5893, 7853)

In [45]:
[word for word, _ in en_vocab.items()][:6], [word for word, _ in de_vocab.items()][:6]

(['<unk>', '<pad>', '<sos>', '<eos>', 'two', 'young'],
 ['<unk>', '<pad>', '<sos>', '<eos>', 'zwei', 'junge'])

In [46]:
en_tokenizer = Tokenizer(WordLevel(en_vocab, unk_token=unk_token))
de_tokenizer = Tokenizer(WordLevel(de_vocab, unk_token=unk_token))

Tokenizer використовується для перетворення вже готових токенів в числа зі словника і навпаки. Якщо такого токену немає в словнику, Tokenizer автоматично замінює його на токен <"unk">

In [47]:
en_tokenizer, de_tokenizer

(Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=None, pre_tokenizer=None, post_processor=None, decoder=None, model=WordLevel(vocab={"<unk>":0, "<pad>":1, "<sos>":2, "<eos>":3, "two":4, "young":5, ",":6, "white":7, "males":8, "are":9, "outside":10, "near":11, "many":12, "bushes":13, ".":14, "several":15, "men":16, "in":17, "hard":18, "hats":19, "operating":20, "a":21, "giant":22, "pulley":23, "system":24, "little":25, "girl":26, "climbing":27, "into":28, "wooden":29, "playhouse":30, "man":31, "blue":32, "shirt":33, "is":34, "standing":35, "on":36, "ladder":37, "cleaning":38, "window":39, "at":40, "the":41, "stove":42, "preparing":43, "food":44, "green":45, "holds":46, "guitar":47, "while":48, "other":49, "observes":50, "his":51, "smiling":52, "stuffed":53, "lion":54, "trendy":55, "talking":56, "her":57, "cellphone":58, "gliding":59, "slowly":60, "down":61, "street":62, "woman":63, "with":64, "large":65, "purse":66, "walking":67, "by":68, "gate":69, "

In [48]:
for word in valid_data["en_tokens"][0]:
    print(en_tokenizer.encode(word).ids)

[2]
[21]
[243]
[74]
[16]
[9]
[1045]
[1249]
[1504]
[21]
[671]
[3]


In [49]:
def numericalize_example(example, en_tokenizer, de_tokenizer):
    en_ids = [en_tokenizer.token_to_id(token) or en_tokenizer.token_to_id("<unk>") for token in example["en_tokens"]]
    de_ids = [de_tokenizer.token_to_id(token) or de_tokenizer.token_to_id("<unk>") for token in example["de_tokens"]]

    return {"en_ids": en_ids, "de_ids": de_ids}

In [50]:
fn_kwargs = {"en_tokenizer": en_tokenizer, "de_tokenizer": de_tokenizer}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [51]:
train_data["en_tokens"][0], train_data["en_ids"][0], [(word, value) for _, (word, value) in enumerate(en_vocab.items())][:15]

(['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 3],
 [('<unk>', 0),
  ('<pad>', 1),
  ('<sos>', 2),
  ('<eos>', 3),
  ('two', 4),
  ('young', 5),
  (',', 6),
  ('white', 7),
  ('males', 8),
  ('are', 9),
  ('outside', 10),
  ('near', 11),
  ('many', 12),
  ('bushes', 13),
  ('.', 14)])

In [52]:
def get_data_loader(data, batch_size, pad_index, shuffle=False):
    max_length = max(
        max(len(seq) for seq in data["en_ids"]),
        max(len(seq) for seq in data["de_ids"])
    )
    
    en_padded = tf.constant(tf.keras.utils.pad_sequences(
        data["en_ids"], 
        padding='post',
        maxlen=max_length,
        value=pad_index
    ))

    de_padded = tf.constant(tf.keras.utils.pad_sequences(
        data["de_ids"],
        padding='post',
        maxlen=max_length,
        value=pad_index
    ))
    
    dataset = tf.data.Dataset.from_tensor_slices({
        "en_ids": en_padded,
        "de_ids": de_padded
    })
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(data))
    
    dataset = dataset.batch(batch_size, drop_remainder=True) # drop_remainder=True відкидає неповний останній батч
    dataset = dataset.prefetch(tf.data.AUTOTUNE) # Використання prefetch для завантаження нового batch під час обробки поточного
    
    return dataset

In [54]:
batch_size = 128
pad_index = en_vocab["<pad>"]

train_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_loader = get_data_loader(valid_data, batch_size, pad_index)
test_loader = get_data_loader(test_data, batch_size, pad_index)

In [55]:
num_batches = 0
for batch in train_loader:
    num_batches += 1

print(f"Number of batches in the train dataset: {num_batches}")

for batch in train_loader.take(1):
    print("en_ids shape:", batch["en_ids"].shape)
    print("de_ids shape:", batch["de_ids"].shape)

Number of batches in the train dataset: 226
en_ids shape: (128, 46)
de_ids shape: (128, 46)


In [56]:
class Encoder(tf.keras.Model):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout):
        super(Encoder, self).__init__() # Щоб отримати доступ до батьківських методів (клас Model)
        self.embedding = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim)
        self.gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=encoder_hidden_dim, return_sequences=True))
        self.dense = tf.keras.layers.Dense(decoder_hidden_dim, activation="tanh")
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, src):
        embedded = self.dropout(self.embedding(src))

        encoder_all_states = self.gru(embedded)
        # encoder_all_states = [batch size, src length, encoder_hidden_dim * 2] - стани для всіх токенів в Bidirectional

        encoder_last_state = encoder_all_states[:, -1, :]
        # encoder_last_state = [batch_size, encoder_hidden_dim * 2]

        encoder_last_state = self.dense(encoder_last_state)
        # encoder_last_state = [batch size, decoder_hidden_dim]

        return encoder_all_states, encoder_last_state

Найкращий механізм уваги для GRU seq2seq - Bahdanau

In [57]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, decoder_hidden_dim):
        super(BahdanauAttention, self).__init__()
        # Обираємо розмірність (decoder_hidden_dim), бо:
        # 1) Ми хочемо, щоб вектори від енкодера і декодера були в одному векторному просторі для обчислення уваги
        # 2) decoder_hidden_dim зазвичай менше, ніж encoder_hidden_dim * 2 (зменшує кількість параметрів)

        self.W1 = tf.keras.layers.Dense(decoder_hidden_dim)
        self.W2 = tf.keras.layers.Dense(decoder_hidden_dim)
        self.V = tf.keras.layers.Dense(1)
    
    def call(self, decoder_hidden_state, encoder_all_states):
        # decoder_hidden_state = [batch size, decoder_hidden_dim]

        decoder_hidden_state = tf.expand_dims(decoder_hidden_state, 1)
        # decoder_hidden_state = [batch_size, 1, decoder_hidden_dim]
        
        score = self.V(tf.nn.tanh(self.W1(encoder_all_states) + self.W2(decoder_hidden_state)))
        # Dense layer застосовується лише до останньої осі
        # self.W1(encoder_all_states) = [batch_size, src_length, decoder_hidden_dim]
        # self.W2(decoder_hidden_state) = [batch_size, 1, decoder_hidden_dim]
        # При додаванні матриць використовується broadcasting для W2 матриці по осі 1
        # score = [batch size, src_length, 1]

        attention_weights = tf.nn.softmax(score, axis=1)
        # Визначення впливу вагів всіх станів на контекстний вектор
        # attention_weights = [batch size, src_length, 1]

        context_vector = attention_weights * encoder_all_states
        # context_vector = [batch size, src_length, encoder_hidden_dim * 2]
        
        context_vector = tf.reduce_sum(context_vector, axis=1)
        # Обчислюємо зважену суму всіх станів
        # context_vector = [batch size, encoder_hidden_dim * 2]

        return context_vector

In [58]:
class Decoder(tf.keras.Model):
    def __init__(self, output_dim, embedding_dim, decoder_hidden_dim, dropout):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.embedding = tf.keras.layers.Embedding(input_dim=output_dim, output_dim=embedding_dim)
        self.gru = tf.keras.layers.GRU(units=decoder_hidden_dim, return_sequences=True)
        self.attention = BahdanauAttention(decoder_hidden_dim)
        self.dense = tf.keras.layers.Dense(output_dim, activation="softmax")
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, input_token, decoder_last_state, encoder_all_states):
        # Перетворює індекс вхідного токена у вектор
        embedded = self.dropout(self.embedding(input_token))
        # embedded = [batch_size, 1, embedding_dim]

        context_vector = self.attention(decoder_last_state, encoder_all_states)
        # context_vector = [batch_size, encoder_hidden_dim * 2]

        context_vector = tf.expand_dims(context_vector, 1)
        # context_vector = [batch_size, 1, encoder_hidden_dim * 2]

        gru_input = tf.concat([embedded, context_vector], axis=-1)
        # gru_input = [batch_size, 1, embedding_dim + encoder_hidden_dim * 2]

        decoder_all_states = self.gru(gru_input, initial_state=decoder_last_state)
        # GRU отримує свій попередній прихований стан, і gru_input (контекстний вектор і новий вхідний токен), і на їх основі обчислює новий прихований стан
        # decoder_all_states = [batch_size, 1, decoder_hidden_dim]

        decoder_new_state = decoder_all_states[:, -1, :]
        # decoder_new_state: [batch_size, decoder_hidden_dim]

        output = self.dense(decoder_all_states)
        # output = [batch_size, 1, output_dim] (output_dim - розмірність словника) 

        return output, decoder_new_state

In [122]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    # На вхід подається два пакета (X batch, y batch)
    def call(self, src, trg, teacher_forcing_ratio):
        # src: Вхідна послідовність (англійський текст) [batch_size, src_len]
        # trg: Вихідна послідовність (німецький текст) [batch_size, trg_len]

        batch_size = tf.shape(src)[0]
        trg_len = tf.shape(trg)[1]
        output_dim = self.decoder.output_dim
        
        # outputs - тензор для зберігання результатів (ймовірності для слів на кожному кроці для одного batch)
        outputs = tf.zeros([batch_size, trg_len, output_dim])

        encoder_all_states, encoder_last_state = self.encoder(src)

        # Першим станом декодеру є останній стан енкодеру
        decoder_current_hidden_state = encoder_last_state

        # Обираємо початковий decoder_input для кожного елементу y пакеті (<sos>)
        decoder_input = tf.expand_dims(trg[:, 0], 1)
        # decoder_input = [batch_size, 1]

        for t in tf.range(1, trg_len):
            step_output, decoder_current_hidden_state = self.decoder(decoder_input, decoder_current_hidden_state, encoder_all_states)
            # step_output - [batch_size, 1, output_dim] (зберігає ймовірності для слів на кроці t)

            outputs = tf.tensor_scatter_nd_update(
                outputs,
                tf.stack([tf.range(batch_size), tf.fill([batch_size], t)], axis=1),
                tf.squeeze(step_output, axis=1)
            )
            # outputs - тензор, який буде оновлюватися
            # tf.range(batch_size): Генерує тензор розміру [batch_size], де кожен елемент — це індекс від 0 до batch_size-1. (буде представляти індекси рядків для кожного елемента в пакеті)
            # tf.fill([batch_size], t) - Створює тензор розміру [batch_size], де всі елементи рівні значенню t (буде представляти індекси стовпців)
            # tf.stack(..., axis=1): Об'єднує ці два тензори вздовж нової осі (вісь 1)
            # tf.squeeze(step_output, axis=1) - видаляє другу вісь (вісь 1)

            teacher_force = tf.random.uniform([]) < teacher_forcing_ratio
            pred_tokens = tf.cast(tf.argmax(step_output, axis=-1), tf.int32)
            # pred_tokens = [batch_size, 1]

            trg_tokens = tf.cast(tf.expand_dims(trg[:, t], 1), tf.int32)
            # trg_tokens = [batch_size, 1]

            decoder_input = tf.where(teacher_force, trg_tokens, pred_tokens)
            # Якщо teacher_force = True, то decoder_input = trg_tokens, інакше decoder_input = pred_tokens

        return outputs

In [123]:
# target: [<sos> word1 word2 <eos> <pad>] - прибираємо перший токен для обрахунку loss
# output: [word1 word2 <eos> <pad> <pad>] - прибираємо останній токен для правильного обрахунку loss
# target_new: [word1 word2 <eos> <pad>]
# output_new: [word1 word2 <eos> <pad>]
# <pad> при обрахунку loss не враховується (masking)

# Функція для тренування 1 batch
# Градієнт (∂loss/∂w) показує, як зміниться loss при зміні ваги w. Це означає, що під час тренування важливо тільки те, як ми обчислюємо loss
def train_step(model, src, trg, optimizer, loss_fn, teacher_forcing_ratio):
    with tf.GradientTape() as tape:
        # Передбачення моделі для одного batch
        output = model(src, trg, teacher_forcing_ratio=teacher_forcing_ratio)
        # output = [batch_size, trg_len, output_dim]
        # trg = [batch_size, trg_len]
        
        # Підготовка даних для обчислення loss
        output = output[:, :-1]
        target = trg[:, 1:]
        
        # Змінюємо форму для обчислення loss
        output_flat = tf.reshape(output, [-1, output.shape[-1]])
        # output_flat = [всі_токени_в_пакеті, output_dim]
        target_flat = tf.reshape(target, [-1])
        # target_flat = [всі_токени_в_пакеті]

        losses = loss_fn(target_flat, output_flat)
        loss = tf.reduce_sum(losses)

    # Обчислюємо градієнти від функції втрат loss щодо всіх тренованих параметрів моделі
    gradients = tape.gradient(loss, model.trainable_variables)

    # Функція tf.clip_by_global_norm використовується для запобігання вибуху градієнтів
    gradients, _ = tf.clip_by_global_norm(gradients, clip_norm=1.0)

    # Оптимізатор оновлює параметри моделі, використовуючи обчислені градієнти
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # zip(gradients, model.trainable_variables) створює пари (градієнт, вага), і оптимізатор застосовує їх для коригування ваг моделі

    # Обчислюємо точність
    predictions = tf.cast(tf.argmax(output_flat, axis=1), tf.int32)
    target_flat = tf.cast(target_flat, tf.int32)

    # Створюємо маску для ігнорування padding токенів
    # target_flat = [5, 8, 1, 1, 3, 1] (1 - pad token)
    # mask буде = [True, True, False, False, True, False]
    # boolean_mask залишає тільки ті елементи, де маска має значення True
    
    mask = tf.math.not_equal(target_flat, en_vocab["<pad>"])

    masked_predictions = tf.boolean_mask(predictions, mask)
    masked_targets = tf.boolean_mask(target_flat, mask)

    accuracy = tf.reduce_mean(tf.cast(tf.equal(masked_predictions, masked_targets), tf.float32))
    
    return loss, accuracy

In [124]:
def evaluate(model, valid_loader, loss_fn):
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    
    for batch in valid_loader:
        src, trg = batch["en_ids"], batch["de_ids"]
        
        # Передбачення моделі (без teacher forcing)
        output = model(src, trg, teacher_forcing_ratio=0.0)
        
        # Підготовка даних для обчислення loss
        output = output[:, :-1]
        target = trg[:, 1:]
        
        output_flat = tf.reshape(output, [-1, output.shape[-1]])
        target_flat = tf.reshape(target, [-1])
        
        losses = loss_fn(target_flat, output_flat)
        loss = tf.reduce_sum(losses)
        
        # Обчислюємо точність
        predictions = tf.cast(tf.argmax(output_flat, axis=1), tf.int32)
        target_flat = tf.cast(target_flat, tf.int32)
        
        mask = tf.math.not_equal(target_flat, en_vocab["<pad>"])
        masked_predictions = tf.boolean_mask(predictions, mask)
        masked_targets = tf.boolean_mask(target_flat, mask)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(masked_predictions, masked_targets), tf.float32))
        
        total_loss += loss
        total_accuracy += accuracy
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches
    
    return avg_loss, avg_accuracy

Під час тренування:
train_loss - обчислюється лише на поточному batch;
valid_loss - обчислюється на всій валідаційній вибірці

In [125]:
def train_model(model, train_loader, valid_loader, optimizer, loss_fn, epochs, teacher_forcing_ratio):
    model.trainable = True
    monitor = CustomTrainingMonitor(log_frequency=10)
    monitor.on_training_start()

    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0
    checkpoint_path = "checkpoints/best_seq2seq_attn_model_weights.weights.h5"
    
    for epoch in range(epochs):
        monitor.on_epoch_start()

        total_train_loss = 0
        total_train_accuracy = 0

        for batch_idx, batch in enumerate(train_loader):
            src, trg = batch["en_ids"], batch["de_ids"]
            
            batch_loss, batch_accuracy = train_step(model, src, trg, optimizer, loss_fn, teacher_forcing_ratio)

            total_train_loss += batch_loss
            total_train_accuracy += batch_accuracy
                
            monitor.on_batch_end(batch_loss, batch_accuracy)
        
        num_batches = batch_idx + 1
        avg_train_loss = total_train_loss / num_batches
        avg_train_accuracy = total_train_accuracy / num_batches
    
        avg_val_loss, avg_val_accuracy = evaluate(model, valid_loader, loss_fn)

        monitor.on_epoch_end(
            avg_train_loss, 
            avg_train_accuracy,
            avg_val_loss,
            avg_val_accuracy
        )

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            model.save_weights(checkpoint_path)
        else:
            patience_counter += 1
            if patience_counter == patience:
                print(f"\nEarly stopping on epoch {epoch+1}")
                break
    
    model.load_weights(checkpoint_path)
    model.save('seq2seq_attn_model.keras')
    monitor.on_training_end()

    trainable_params = sum(tf.size(v) for v in model.trainable_variables)
    non_trainable_params = sum(tf.size(v) for v in model.non_trainable_variables)
    total_params = trainable_params + non_trainable_params

    print("\nModel Parameters Summary:")
    print(f"Trainable params: {trainable_params:,}")
    print(f"Non-trainable params: {non_trainable_params:,}")
    print(f"Total params: {total_params:,}")

In [127]:
def create_model():
    encoder = Encoder(input_dim=len(en_vocab), 
                     embedding_dim=128,
                     encoder_hidden_dim=128,
                     decoder_hidden_dim=128,
                     dropout=0.2)
    decoder = Decoder(output_dim=len(de_vocab), 
                     embedding_dim=128, 
                     decoder_hidden_dim=128, 
                     dropout=0.2)
    
    optimizer = tf.keras.optimizers.Adam(0.001)

    return Seq2Seq(encoder, decoder), optimizer

In [128]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE, ignore_class=en_vocab["<pad>"])
epochs = 3
teacher_forcing_ratio=0.5

In [129]:
seq2seq_model, optimizer = create_model()
train_model(seq2seq_model, train_loader, valid_loader, optimizer, loss_fn, epochs, teacher_forcing_ratio)

Running training...

Epoch 1
Epoch start: TensorFlow reported memory: 410MB (Peak: 3585MB)
Batch 10 - Loss: 13216.4678 - Accuracy: 0.0762
TensorFlow reported memory: 461MB (Peak: 3585MB)
Batch 20 - Loss: 11538.7812 - Accuracy: 0.0747
TensorFlow reported memory: 461MB (Peak: 3585MB)
Batch 30 - Loss: 10453.6914 - Accuracy: 0.0743
TensorFlow reported memory: 461MB (Peak: 3585MB)
Batch 40 - Loss: 10284.2500 - Accuracy: 0.0739
TensorFlow reported memory: 461MB (Peak: 3585MB)
Batch 50 - Loss: 10028.8594 - Accuracy: 0.0750
TensorFlow reported memory: 461MB (Peak: 3585MB)
Batch 60 - Loss: 10031.8359 - Accuracy: 0.0788
TensorFlow reported memory: 187MB (Peak: 3585MB)
Batch 70 - Loss: 10475.5762 - Accuracy: 0.0856
TensorFlow reported memory: 187MB (Peak: 3585MB)
Batch 80 - Loss: 10409.7383 - Accuracy: 0.0944
TensorFlow reported memory: 187MB (Peak: 3585MB)
Batch 90 - Loss: 9933.1768 - Accuracy: 0.1093
TensorFlow reported memory: 187MB (Peak: 3585MB)
Batch 100 - Loss: 9769.0898 - Accuracy: 0.0981

In [130]:
avg_loss, avg_accuracy = evaluate(seq2seq_model, test_loader, loss_fn)
print("Evaluating on test data...")
print(f"Test Loss: {avg_loss:.4f} - Test Accuracy: {avg_accuracy:.4f}")

Evaluating on test data...
Test Loss: 8889.0928 - Test Accuracy: 0.1492


top-1 accuracy буде низьким, якщо правильний клас не має найвищої ймовірності у багатьох випадках

In [131]:
def translate_sentence(sentence, model, en_nlp, en_vocab, de_vocab, sos_token, eos_token, max_output_length=50):
    model.trainable = False
    id_to_token = {id: token for token, id in de_vocab.items()}
    
    if isinstance(sentence, str):
        en_tokens = [token.text for token in en_nlp.tokenizer(sentence)]
    else:
        en_tokens = [token for token in sentence]
    
    en_tokens = [token.lower() for token in en_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    
    en_ids = [en_vocab[token] if token in en_vocab else en_vocab["<unk>"] for token in en_tokens]
    
    src = tf.expand_dims(en_ids, 0) 
    # src = [1, src_len]

    decoder_input = tf.constant([[de_vocab[sos_token]]])

    encoder_all_states, encoder_last_state = model.encoder(src, training=False)

    decoder_current_hidden_state = encoder_last_state
    
    output_tokens = [sos_token]
    
    for _ in range(max_output_length):
        step_output, decoder_current_hidden_state = model.decoder(
            decoder_input,
            decoder_current_hidden_state,
            encoder_all_states,
            training=False
        )

        predicted_id = int(tf.argmax(step_output, axis=-1))
        
        predicted_token = id_to_token[predicted_id]

        output_tokens.append(predicted_token)
        
        if predicted_token == eos_token:
            break

        decoder_input = tf.constant([[predicted_id]])
    
    return en_tokens, output_tokens

In [132]:
for i in range (5):
    en_tokens, output_tokens = translate_sentence(test_data["en"][i], seq2seq_model, en_nlp, en_vocab, de_vocab, sos_token, eos_token)
    print("Input: " + " ".join(en_tokens))
    print("Output: " + " ".join(test_data["de_tokens"][i]))
    print("Predictions: " + " ".join(output_tokens) + '\n')

Input: <sos> a man in an orange hat starring at something . <eos>
Output: <sos> ein mann mit einem orangefarbenen hut , der etwas anstarrt . <eos>
Predictions: <sos> mann in einem einem einem einem <unk> . <eos>

Input: <sos> a boston terrier is running on lush green grass in front of a white fence . <eos>
Output: <sos> ein boston terrier läuft über saftig-grünes gras vor einem weißen zaun . <eos>
Predictions: <sos> mann in einem einem einem , einem einem , . . . <eos>

Input: <sos> a girl in karate uniform breaking a stick with a front kick . <eos>
Output: <sos> ein mädchen in einem karateanzug bricht ein brett mit einem tritt . <eos>
Predictions: <sos> mann in einem einem einem , einem einem , . . <eos>

Input: <sos> five people wearing winter jackets and helmets stand in the snow , with snowmobiles in the background . <eos>
Output: <sos> fünf leute in winterjacken und mit helmen stehen im schnee mit schneemobilen im hintergrund . <eos>
Predictions: <sos> mann in einem einem und eine